In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
from time import sleep
from random import randint
from datetime import datetime
import pprint

In [2]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
}

def get_url(position, location):

    '''
    parses indeed and returns the (position, location)

    '''
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

def get_record(card):
    '''
    Extract job data from a single record and returns job title, company, location, and url 

    '''
    atag = card.h2.a
    try:
        job_title = atag.get('title')
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'company').text.strip()
    except AttributeError:
        company = ''
    try:
        location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except AttributeError:
        location = ''
    
    
    extract_date = datetime.today().strftime('%Y-%m-%d')
    job_url = 'https://www.indeed.com' + atag.get('href')
    
    return (job_title, company, location, job_url)

def main(position, location, limit=1):
    # Run the main program reouting
    records = []  # creating the record list
    url = get_url(position, location)  # create the url while passing in the position and location.
    
    while True:
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        
        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            #delay = randint(1, 10)
            #sleep(delay)
        except AttributeError:
            break
         
    with open('Indeed.csv', 'w', newline='', encoding='utf-8') as f:
       writer = csv.writer(f)
       writer.writerow(['Job Title', 'Company', 'Location', 'Job Url'])
       writer.writerows(records)

main('python', 'New York')

https://www.indeed.com/jobs?q=python&l=New York
https://www.indeed.com/jobs?q=python&l=New+York&start=10
https://www.indeed.com/jobs?q=python&l=New+York&start=20
https://www.indeed.com/jobs?q=python&l=New+York&start=30
https://www.indeed.com/jobs?q=python&l=New+York&start=40
https://www.indeed.com/jobs?q=python&l=New+York&start=50
https://www.indeed.com/jobs?q=python&l=New+York&start=60
https://www.indeed.com/jobs?q=python&l=New+York&start=70
https://www.indeed.com/jobs?q=python&l=New+York&start=80
https://www.indeed.com/jobs?q=python&l=New+York&start=90
https://www.indeed.com/jobs?q=python&l=New+York&start=100
https://www.indeed.com/jobs?q=python&l=New+York&start=110
https://www.indeed.com/jobs?q=python&l=New+York&start=120
https://www.indeed.com/jobs?q=python&l=New+York&start=130
https://www.indeed.com/jobs?q=python&l=New+York&start=140
https://www.indeed.com/jobs?q=python&l=New+York&start=150
https://www.indeed.com/jobs?q=python&l=New+York&start=160
https://www.indeed.com/jobs?q=pyt

In [3]:
pd.read_csv('Indeed.csv')

,Job Title,Company,Location,Job Url
0,Python Instructor,Code Immersives NYC,"New York, NY",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Python software Engineer,emids,New York State,https://www.indeed.com/rc/clk?jk=d3f0300c3316d...
2,Looking for an entry-level or graduate job?,Unibuddy,"New York, NY",https://www.indeed.com/rc/clk?jk=6f71dbba16836...
3,Python Developer (Initial Remote till Covid-19),"Xyant Technology, Inc.","New York, NY",https://www.indeed.com/rc/clk?jk=573d1be1f8df7...
4,Python Software Engineer for Athena Trade & Risk,"JPMorgan Chase Bank, N.A.","New York, NY",https://www.indeed.com/rc/clk?jk=cce0aaf678bb2...
...,...,...,...,...
1055,"Actuarial Manager, Private Client Group",AIG,"New York, NY",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1056,Senior Software Engineer - Market Data SRE,Bloomberg,"New York, NY",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1057,C++ Developer,The Execu|Search Group,"New York, NY",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1058,Project Manager with RHEL expertise - Japanese...,"Pasona N A, Inc.","New York, NY",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
